In [1]:
import langchain, langchain_core, langchain_community
from langchain_groq import ChatGroq
import os

In [2]:
llm = ChatGroq(
    temperature=0, model_name="gemma2-9b-it", api_key=os.environ.get("GROQ_KEY")
)

NEO4J_URI = os.environ.get("NEO4J_URI")
NEO4J_USERNAME = os.environ.get("NEO4J_USERNAME")
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")

In [3]:
from langchain_community.graphs import Neo4jGraph

langchain.verbose = True
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [4]:
from langchain_core.documents import Document

text = """
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and receivedBachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents = [Document(page_content=text)]

In [5]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [6]:
print(graph_documents[0].nodes)

[Node(id='Elon Reeve Musk', type='Person'), Node(id='Spacex', type='Company'), Node(id='Tesla, Inc.', type='Company'), Node(id='X Corp.', type='Company'), Node(id='Twitter', type='Company'), Node(id='The Boring Company', type='Company'), Node(id='Xai', type='Company'), Node(id='Neuralink', type='Company'), Node(id='Openai', type='Company'), Node(id='Forbes', type='Organization'), Node(id='Pretoria', type='City'), Node(id='Canada', type='Country'), Node(id='University Of Pretoria', type='University'), Node(id="Queen'S University", type='University'), Node(id='University Of Pennsylvania', type='University'), Node(id='California', type='State'), Node(id='Stanford University', type='University'), Node(id='Zip2', type='Company')]


In [8]:
movie_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [10]:
graph
graph.query(movie_query)

graph.refresh_schema()
print(graph.schema)

Node properties:
Person {name: STRING, birthday: STRING}
Movie {title: STRING, year: INTEGER, id: STRING, released: DATE, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [11]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True)


In [12]:
response = chain.invoke({"query": "Who was the director of the moview GoldenEye"})




> Entering new GraphCypherQAChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
Node properties are the following:
Person {name: STRING, birthday: STRING},Movie {title: STRING, year: INTEGER, id: STRING, released: DATE, imdbRating: FLOAT},Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie),(:Movie)-[:IN_GENRE]->(:Genre)
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
Who was the director of the moview GoldenEye

> Finis

In [13]:
print(response)

{'query': 'Who was the director of the moview GoldenEye', 'result': 'Martin Campbell  \n'}
